<a href="https://colab.research.google.com/github/anudeeplearning/Market-Risk/blob/main/hybrid_var.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install yfinance

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 6.4 MB 11.8 MB/s 
     |████████████████████████████████| 63 kB 1.9 MB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
  Attempting uninstall: lxml
    Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
import yfinance as yf, numpy as np, pandas as pd, datetime as dt, matplotlib.pyplot as plt, scipy.stats as scs

In [ ]:
tickers = ['AAPL', 'FB', 'C','DIS','F','MSFT','MS','GME','TSLA','AMZN']
weights = np.array(10*[0.1])
lamda=.94

In [ ]:
start = dt.datetime(2019,1,1)
end=dt.datetime.now()

df=yf.download(tickers,start,end)['Adj Close']
df

In [ ]:
returns = df.pct_change()

In [ ]:
portfolio_returns = returns @ weights

In [ ]:
df = portfolio_returns.to_frame()

In [ ]:
days_size = len(portfolio_returns)
df['days'] = np.array(range(days_size))

In [ ]:
ewma = [0]*days_size
for i in range(days_size):
  ewma[i]=(1-lamda)*(lamda)**i

In [ ]:
df

,0,days
Date,,
2020-03-16,-0.123834,302
2020-03-12,-0.104239,300
2020-03-09,-0.090666,297
2020-06-11,-0.077275,363
2020-03-18,-0.067712,304
...,...,...
2020-03-13,0.110393,301
2021-01-27,0.112020,521
2021-02-24,0.120245,540


In [ ]:
ewma

In [ ]:
df=df.sort_values(by=[0])

In [ ]:
df

,0,days,hybrid
Date,,,
2020-03-16,-0.123834,302,-1.497475e-10
2020-03-12,-0.104239,300,-1.186028e-10
2020-03-09,-0.090666,297,-9.415033e-11
2020-06-11,-0.077275,363,-5.990783e-10
2020-03-18,-0.067712,304,-8.702420e-11
...,...,...,...
2020-03-13,0.110393,301,1.294888e-10
2021-01-27,0.112020,521,1.068581e-07
2021-02-24,0.120245,540,2.046050e-07


In [ ]:
days_size

862

In [ ]:
len(df)

862

In [ ]:
df['hybrid'] = np.array([0.1]*days_size)

In [ ]:
df.iloc[1,1]=1

In [ ]:
for i in range(days_size):
  numb = days_size - df.iloc[i,1]-1
  df.iloc[i,2] = float(ewma[numb])

In [ ]:
df['cumm_hybrid'] = [0.1]*days_size
df.iloc[0,3]=df.iloc[0,2]

In [ ]:
for i in range(1,days_size):
  numb = df.iloc[i,2]
  df.iloc[i,3] = df.iloc[i,2]+df.iloc[i-1,3]

In [ ]:
df

,0,days,hybrid,cumm_hybrid
Date,,,,
2020-03-16,-0.123834,302,5.709807e-17,5.709807e-17
2020-03-12,-0.104239,300,5.045185e-17,1.075499e-16
2020-03-09,-0.090666,297,4.190450e-17,1.494544e-16
2020-06-11,-0.077275,363,2.487839e-15,2.637294e-15
2020-03-18,-0.067712,304,6.461981e-17,2.701914e-15
...,...,...,...,...
2020-03-13,0.110393,301,5.367218e-17,1.000000e+00
2021-01-27,0.112020,521,4.381486e-11,1.000000e+00
2021-02-24,0.120245,540,1.419686e-10,1.000000e+00


In [ ]:
confidence_interval = 0.01

In [ ]:
ans=0

In [ ]:
for i in range(days_size):
  if df.iloc[i,3]>=confidence_interval:
    print (df.iloc[i,0])
    ans=df.iloc[i,0]
    break

-0.05480138092457617


In [ ]:
num_days=10
var_reg = ans*(num_days)**(1/2)
var_reg

-0.1732971826441648

The regulary VaR using hybrid method is calculated as 99%VaR is 10*1day 99%VaR which is 17.32 when compared to 16.48% calculated from historical simulation method. Hybrid VaR reacts better to market changes when compared to historical VaR and thus has better performance and lower number of violations